# Chapter 2

Program included: context-free phrase-structure grammar

In [5]:
;; imported function
(defun mappend (fn the-list)
    "Apply fn to the list and append the results"
    (apply #'append (mapcar fn the-list)))

MAPPEND

In [6]:
(defun one-of (set)
    "Pick one element of set, and make a list of it."
    (list (random-elt set)))

(defun random-elt (choices)
    "Choose an element from a list at random."
    (elt choices (random (length choices))))

ONE-OF

RANDOM-ELT

SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::RANDOM-ELT


In [7]:
(defun sentence () (append (noun-phrase) (verb-phrase)))
(defun noun-phrase () (append (Article) (Noun)))
(defun verb-phrase () (append (Verb) (noun-phrase)))
(defun Article () (one-of '(the a)))
(defun Noun () (one-of '(man ball woman table)))
(defun Verb () (one-of '(hit took saw liked)))

SENTENCE

NOUN-PHRASE

VERB-PHRASE

ARTICLE

NOUN

VERB

SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::NOUN-PHRASE
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::VERB-PHRASE
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::ARTICLE
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::NOUN
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::VERB


In [8]:
(sentence)

(A BALL SAW A TABLE)

In [9]:
;; expand the rule to include Kleene star
(defun noun-phrase () (append (Article) (Adj*) (Noun) (PP*)))
(defun Adj* ()
    (if (= (random 2) 0)
        nil
        (append (Adj) (Adj*))))
(defun PP* ()
    (if (= (random 2) 0)
        nil
        (append (Prep) (PP*))))
(defun Adj () (one-of '(big little blue green red)))
(defun Prep () (one-of '(to in by with)))

NOUN-PHRASE

ADJ*

PP*

ADJ

PREP

SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::ADJ*
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::PP*
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::NOUN-PHRASE in DEFUN
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::ADJ
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::PREP


In [10]:
(sentence)

(A WOMAN TO TO BY IN WITH SAW THE RED RED LITTLE WOMAN IN WITH)

In [11]:
;;; Not a scalable solution, change to rule-based solution

(defparameter *simple-grammar*
    '((sentence -> (noun-phrase verb-phrase))
      (noun-phrase -> (Article Noun))
      (verb-phrase -> (Verb noun-phrase))
      (Article -> the a)
      (Noun -> man ball woman table)
      (Verb -> hit took saw liked))
    "A grammar for a trivial subset of English.")

;; Add a layer of indirection, boundary for what a program could change
(defvar *grammar* *simple-grammar*
    "The grammar used by generate. Initially, this is *simple-grammar*, but we can switch to other grammars.")

*SIMPLE-GRAMMAR*

*GRAMMAR*

In [12]:
(assoc 'noun *grammar*)

(NOUN -> MAN BALL WOMAN TABLE)

In [13]:
;; define procedures to operate on the grammar
(defun rule-lhs (rule) (first rule))
(defun rule-rhs (rule) (rest (rest rule)))
(defun rewrites (category) 
    "Return a list of possible rewrites for this category"
    (rule-rhs (assoc category *grammar*)))

RULE-LHS

RULE-RHS

REWRITES

In [14]:
;; Exercise 2.1, generate
(defun generate (phrase)
    (let ((choices (rewrites phrase)))
        (cond ((listp phrase) (mappend #'generate phrase))
            (choices (generate (random-elt choices)))
            (t (list phrase)))))

(generate 'sentence)

GENERATE

(THE BIG ADIABATIC GREEN WOMAN ON IT LIKED SHE)

In [15]:
;; Exercise 2.2, generate differentiates terminal and non-terminal symbols
(defun generate2 (phrase)
    (if (listp phrase) 
        (mappend #'generate2 phrase)
        (if (null (rewrites phrase))
                    (list phrase)
                    (generate2 (random-elt (rewrites phrase))))))

(generate2 'sentence)

GENERATE2

(THE MAN TOOK NAME BY NAME WITH SHE)

In [16]:
;; Data-driven advantage 1: the program could be easily extended to more complext grammar
(defparameter *bigger-grammar*
    '((sentence -> (noun-phrase verb-phrase))
      (noun-phrase -> (Article Adj* Noun PP*) (Name) (Pronoun))
      (verb-phrase -> (Verb noun-phrase PP*))
      (Adj* -> () (Adj Adj*))
      (PP* -> () (PP PP*))
      (PP -> (Prep noun-phrase))
      (Prep -> to in by with on)
      (Adj -> big little blue green adiabatic)
      (Article -> the a)
      (Noun -> man ball woman table)
      (Verb -> hit took saw liked)
      (pronoun -> he she it these those that))
    "A more complext grammar.")

;; Demostration the value of indirection layer
(setf *grammar* *bigger-grammar*)

*BIGGER-GRAMMAR*

((SENTENCE -> (NOUN-PHRASE VERB-PHRASE))
 (NOUN-PHRASE -> (ARTICLE ADJ* NOUN PP*) (NAME) (PRONOUN))
 (VERB-PHRASE -> (VERB NOUN-PHRASE PP*)) (ADJ* -> NIL (ADJ ADJ*))
 (PP* -> NIL (PP PP*)) (PP -> (PREP NOUN-PHRASE)) (PREP -> TO IN BY WITH ON)
 (ADJ -> BIG LITTLE BLUE GREEN ADIABATIC) (ARTICLE -> THE A)
 (NOUN -> MAN BALL WOMAN TABLE) (VERB -> HIT TOOK SAW LIKED)
 (PRONOUN -> HE SHE IT THESE THOSE THAT))

In [50]:
;; Data-driven advantage 2: write another program working on same data is trivil
(defun generate-tree (phrase)
    "Generate a random sentence or phrase, with a complete parse tree."
    (if (listp phrase) 
        (mapcar #'generate-tree phrase)
        (if (null (rewrites phrase))
                    (list phrase)
                    (cons phrase (generate-tree (random-elt (rewrites phrase)))))))

(generate-tree 'sentence)

GENERATE-TREE

(SENTENCE (NOUN-PHRASE (ARTICLE THE) (NOUN BALL))
          (VERB-PHRASE (VERB TOOK) (NOUN-PHRASE (ARTICLE A) (NOUN BALL))))

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::GENERATE-TREE in DEFUN


In [62]:
(defun generate-all (phrase)
    "Return a list of list with all the possible phrases."
    (cond ((null phrase) '(nil))
          ((listp phrase) 
            (combine-all (generate-all (first phrase)) (generate-all (rest phrase))))
          ((rewrites phrase)
            (mappend #'generate-all (rewrites phrase)))
          (t (list (list phrase)))))

(defun combine-all (xlist ylist)
    (mappend #'(lambda (y) 
                (mapcar #'(lambda (x) (append x y)) xlist)) ylist))

(setf *grammar* *simple-grammar*)
(length (generate-all 'sentence))

GENERATE-ALL

COMBINE-ALL

((SENTENCE -> (NOUN-PHRASE VERB-PHRASE)) (NOUN-PHRASE -> (ARTICLE NOUN))
 (VERB-PHRASE -> (VERB NOUN-PHRASE)) (ARTICLE -> THE A)
 (NOUN -> MAN BALL WOMAN TABLE) (VERB -> HIT TOOK SAW LIKED))

256

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::GENERATE-ALL in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::COMBINE-ALL in DEFUN


In [77]:
;; Exercise 2.4
(defun cross-product (fn xlist ylist)
    (mappend #'(lambda (y) (mapcar #'(lambda (x) (funcall fn x y)) xlist)) ylist))

(defun combine-all (xlist ylist)
  "Return a list of lists formed by appending a y to an x"
  (cross-product #'append xlist ylist))

(cross-product #'append '((1) (2) (3)) '((a) (b) (c)))
(combine-all '((1) (2) (3)) '((a) (b) (c)))

CROSS-PRODUCT

COMBINE-ALL

((1 A) (2 A) (3 A) (1 B) (2 B) (3 B) (1 C) (2 C) (3 C))

((1 A) (2 A) (3 A) (1 B) (2 B) (3 B) (1 C) (2 C) (3 C))

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::CROSS-PRODUCT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::COMBINE-ALL in DEFUN
